# Import libraries

In [17]:
import pandas as pd
from pandas import DataFrame
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sb
import time

from sklearn.model_selection import KFold
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import train_test_split

# Import data

In [2]:
train = pd.read_csv(r"C:\Users\Senthil\OneDrive\DSM_projects\06-Kaggle_projects\NGS_Bacteria\train_dataset.csv")

In [3]:
train.head()

,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,...,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0,target
0,197019,4.630000e-08,2.460000e-06,3.080000e-06,0.000063,0.000150,1.296740e-04,0.000110,0.000074,0.000003,...,0.000040,0.000030,0.000043,0.000055,0.000028,0.000001,0.000005,9.460000e-06,4.630000e-08,Salmonella_enterica
1,28056,-9.540000e-07,-5.540000e-06,3.080000e-06,0.000048,0.000087,3.870000e-05,0.000055,0.000041,-0.000009,...,0.000116,0.000068,0.000142,0.000125,0.000093,0.000009,0.000012,3.250000e-05,4.630000e-08,Escherichia_coli
2,8210,-9.540000e-07,1.050000e-05,1.710000e-05,-0.000004,-0.000020,7.970000e-05,-0.000030,0.000036,-0.000043,...,0.000054,0.000097,0.000064,0.000114,-0.000003,-0.000010,0.000021,4.630000e-07,-9.540000e-07,Streptococcus_pneumoniae
3,98476,4.630000e-08,1.460000e-06,8.470000e-08,0.000008,0.000030,2.970000e-05,0.000036,0.000022,0.000003,...,0.000048,0.000025,0.000039,0.000042,0.000019,0.000004,0.000002,4.460000e-06,4.630000e-08,Escherichia_coli
4,166342,-9.540000e-07,4.630000e-07,-1.290000e-05,0.000036,0.000060,-3.260000e-07,-0.000040,0.000066,-0.000033,...,0.000134,0.000087,0.000134,0.000134,0.000037,0.000010,0.000010,4.050000e-05,-9.540000e-07,Escherichia_fergusonii


In [4]:
test = pd.read_csv(r"C:\Users\Senthil\OneDrive\DSM_projects\06-Kaggle_projects\NGS_Bacteria\test_dataset.csv")

In [5]:
test.head()

,row_id,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,...,A8T0G0C2,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0
0,86283,-9.540000e-07,-9.540000e-06,-2.290000e-05,0.000086,-0.000010,-0.000010,0.000100,0.000026,-0.000033,...,0.000047,0.000174,0.000117,0.000144,0.000204,0.000167,0.000030,4.630000e-07,0.000030,-9.540000e-07
1,158141,-9.540000e-07,4.630000e-07,-2.290000e-05,-0.000024,-0.000100,-0.000070,-0.000140,-0.000034,-0.000043,...,0.000087,0.000444,0.000297,0.000264,0.000454,0.000177,0.000071,8.050000e-05,0.000081,-9.540000e-07
2,104780,-9.540000e-07,-9.540000e-06,3.710000e-05,0.000436,0.001160,0.001710,0.001230,0.000566,0.000087,...,0.000017,0.000014,0.000067,0.000084,0.000014,0.000017,0.000030,2.050000e-05,0.000021,-9.540000e-07
3,87545,-9.540000e-07,-8.540000e-06,-4.090000e-05,-0.000108,-0.000196,-0.000237,-0.000192,-0.000113,-0.000040,...,0.000305,0.000739,0.000460,0.000796,0.000859,0.000523,0.000117,1.624630e-04,0.000129,4.630000e-08
4,174958,4.630000e-08,-2.540000e-06,-9.150000e-07,0.000008,0.000040,0.000009,0.000021,0.000019,-0.000009,...,0.000028,0.000077,0.000050,0.000107,0.000112,0.000064,0.000007,6.460000e-06,0.000025,4.630000e-08


# Understanding the data

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24000 entries, 0 to 23999
Columns: 288 entries, row_id to target
dtypes: float64(286), int64(1), object(1)
memory usage: 52.7+ MB


In [7]:
print(train.shape)

(24000, 288)


In [8]:
print(test.shape)

(6000, 287)


In [9]:
train.isnull().sum().sort_values(ascending=False)

row_id       0
A0T0G0C10    0
A3T4G3C0     0
A3T4G2C1     0
A3T4G1C2     0
            ..
A1T3G1C5     0
A1T3G0C6     0
A1T2G7C0     0
A1T2G6C1     0
target       0
Length: 288, dtype: int64

# drop row ID from both train and test

In [10]:
train=train.drop(['row_id'],axis=1)
train.head()

,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,A0T0G9C1,...,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0,target
0,4.630000e-08,2.460000e-06,3.080000e-06,0.000063,0.000150,1.296740e-04,0.000110,0.000074,0.000003,-0.000007,...,0.000040,0.000030,0.000043,0.000055,0.000028,0.000001,0.000005,9.460000e-06,4.630000e-08,Salmonella_enterica
1,-9.540000e-07,-5.540000e-06,3.080000e-06,0.000048,0.000087,3.870000e-05,0.000055,0.000041,-0.000009,-0.000007,...,0.000116,0.000068,0.000142,0.000125,0.000093,0.000009,0.000012,3.250000e-05,4.630000e-08,Escherichia_coli
2,-9.540000e-07,1.050000e-05,1.710000e-05,-0.000004,-0.000020,7.970000e-05,-0.000030,0.000036,-0.000043,-0.000010,...,0.000054,0.000097,0.000064,0.000114,-0.000003,-0.000010,0.000021,4.630000e-07,-9.540000e-07,Streptococcus_pneumoniae
3,4.630000e-08,1.460000e-06,8.470000e-08,0.000008,0.000030,2.970000e-05,0.000036,0.000022,0.000003,-0.000008,...,0.000048,0.000025,0.000039,0.000042,0.000019,0.000004,0.000002,4.460000e-06,4.630000e-08,Escherichia_coli
4,-9.540000e-07,4.630000e-07,-1.290000e-05,0.000036,0.000060,-3.260000e-07,-0.000040,0.000066,-0.000033,-0.000010,...,0.000134,0.000087,0.000134,0.000134,0.000037,0.000010,0.000010,4.050000e-05,-9.540000e-07,Escherichia_fergusonii


In [11]:
test=test.drop(['row_id'],axis=1)
test.head()

,A0T0G0C10,A0T0G1C9,A0T0G2C8,A0T0G3C7,A0T0G4C6,A0T0G5C5,A0T0G6C4,A0T0G7C3,A0T0G8C2,A0T0G9C1,...,A8T0G0C2,A8T0G1C1,A8T0G2C0,A8T1G0C1,A8T1G1C0,A8T2G0C0,A9T0G0C1,A9T0G1C0,A9T1G0C0,A10T0G0C0
0,-9.540000e-07,-9.540000e-06,-2.290000e-05,0.000086,-0.000010,-0.000010,0.000100,0.000026,-0.000033,-0.000010,...,0.000047,0.000174,0.000117,0.000144,0.000204,0.000167,0.000030,4.630000e-07,0.000030,-9.540000e-07
1,-9.540000e-07,4.630000e-07,-2.290000e-05,-0.000024,-0.000100,-0.000070,-0.000140,-0.000034,-0.000043,-0.000010,...,0.000087,0.000444,0.000297,0.000264,0.000454,0.000177,0.000071,8.050000e-05,0.000081,-9.540000e-07
2,-9.540000e-07,-9.540000e-06,3.710000e-05,0.000436,0.001160,0.001710,0.001230,0.000566,0.000087,-0.000010,...,0.000017,0.000014,0.000067,0.000084,0.000014,0.000017,0.000030,2.050000e-05,0.000021,-9.540000e-07
3,-9.540000e-07,-8.540000e-06,-4.090000e-05,-0.000108,-0.000196,-0.000237,-0.000192,-0.000113,-0.000040,-0.000010,...,0.000305,0.000739,0.000460,0.000796,0.000859,0.000523,0.000117,1.624630e-04,0.000129,4.630000e-08
4,4.630000e-08,-2.540000e-06,-9.150000e-07,0.000008,0.000040,0.000009,0.000021,0.000019,-0.000009,-0.000008,...,0.000028,0.000077,0.000050,0.000107,0.000112,0.000064,0.000007,6.460000e-06,0.000025,4.630000e-08


# Installation of lazypredict

In [12]:
!pip install lazypredict

In [14]:
from lazypredict.Supervised import LazyClassifier

# Define X and y

In [13]:
y = train.pop('target')
X = train

In [18]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42,shuffle=True, stratify=y)

# Model training

In [19]:
clf = LazyClassifier(verbose=0,predictions=True)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [14:26<00:00, 29.88s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LGBMClassifier,0.92,0.92,None,0.92,33.93
RandomForestClassifier,0.92,0.92,None,0.92,14.48
ExtraTreesClassifier,0.91,0.91,None,0.91,8.01
KNeighborsClassifier,0.87,0.87,None,0.87,3.35
SVC,0.87,0.87,None,0.87,83.85
BaggingClassifier,0.85,0.85,None,0.85,20.46
LogisticRegression,0.85,0.85,None,0.85,3.74
LinearSVC,0.81,0.81,None,0.81,81.66
CalibratedClassifierCV,0.80,0.80,None,0.80,316.20


In [20]:
predictions.head()

,AdaBoostClassifier,BaggingClassifier,BernoulliNB,CalibratedClassifierCV,DecisionTreeClassifier,DummyClassifier,ExtraTreeClassifier,ExtraTreesClassifier,GaussianNB,KNeighborsClassifier,...,NuSVC,PassiveAggressiveClassifier,Perceptron,QuadraticDiscriminantAnalysis,RandomForestClassifier,RidgeClassifier,RidgeClassifierCV,SGDClassifier,SVC,LGBMClassifier
0,Staphylococcus_aureus,Bacteroides_fragilis,Bacteroides_fragilis,Bacteroides_fragilis,Bacteroides_fragilis,Bacteroides_fragilis,Bacteroides_fragilis,Bacteroides_fragilis,Streptococcus_pneumoniae,Bacteroides_fragilis,...,Bacteroides_fragilis,Bacteroides_fragilis,Bacteroides_fragilis,Streptococcus_pneumoniae,Bacteroides_fragilis,Bacteroides_fragilis,Bacteroides_fragilis,Bacteroides_fragilis,Bacteroides_fragilis,Bacteroides_fragilis
1,Campylobacter_jejuni,Staphylococcus_aureus,Staphylococcus_aureus,Staphylococcus_aureus,Staphylococcus_aureus,Bacteroides_fragilis,Streptococcus_pneumoniae,Staphylococcus_aureus,Staphylococcus_aureus,Staphylococcus_aureus,...,Staphylococcus_aureus,Staphylococcus_aureus,Staphylococcus_aureus,Staphylococcus_aureus,Staphylococcus_aureus,Staphylococcus_aureus,Staphylococcus_aureus,Staphylococcus_aureus,Staphylococcus_aureus,Staphylococcus_aureus
2,Staphylococcus_aureus,Streptococcus_pneumoniae,Bacteroides_fragilis,Streptococcus_pneumoniae,Streptococcus_pneumoniae,Bacteroides_fragilis,Streptococcus_pneumoniae,Streptococcus_pneumoniae,Streptococcus_pneumoniae,Streptococcus_pneumoniae,...,Streptococcus_pneumoniae,Streptococcus_pneumoniae,Streptococcus_pneumoniae,Streptococcus_pneumoniae,Streptococcus_pneumoniae,Streptococcus_pneumoniae,Streptococcus_pneumoniae,Streptococcus_pneumoniae,Streptococcus_pneumoniae,Streptococcus_pneumoniae
3,Campylobacter_jejuni,Enterococcus_hirae,Enterococcus_hirae,Enterococcus_hirae,Enterococcus_hirae,Bacteroides_fragilis,Enterococcus_hirae,Enterococcus_hirae,Staphylococcus_aureus,Enterococcus_hirae,...,Enterococcus_hirae,Enterococcus_hirae,Streptococcus_pneumoniae,Enterococcus_hirae,Enterococcus_hirae,Enterococcus_hirae,Enterococcus_hirae,Enterococcus_hirae,Enterococcus_hirae,Enterococcus_hirae
4,Enterococcus_hirae,Bacteroides_fragilis,Escherichia_fergusonii,Bacteroides_fragilis,Bacteroides_fragilis,Bacteroides_fragilis,Bacteroides_fragilis,Bacteroides_fragilis,Escherichia_fergusonii,Bacteroides_fragilis,...,Bacteroides_fragilis,Streptococcus_pneumoniae,Escherichia_fergusonii,Streptococcus_pneumoniae,Bacteroides_fragilis,Bacteroides_fragilis,Bacteroides_fragilis,Bacteroides_fragilis,Bacteroides_fragilis,Bacteroides_fragilis


# Model Performance

In [21]:
from sklearn.metrics import classification_report
for i in predictions.columns.tolist():
    print('\t\t',i,'\n')
    print(classification_report(y_test, predictions[i]),'\n')

		 AdaBoostClassifier 

                          precision    recall  f1-score   support

    Bacteroides_fragilis       0.38      0.30      0.33       491
    Campylobacter_jejuni       0.29      0.75      0.42       489
      Enterococcus_hirae       0.29      0.22      0.25       486
        Escherichia_coli       0.33      0.14      0.20       488
  Escherichia_fergusonii       0.47      0.64      0.54       485
   Klebsiella_pneumoniae       0.80      0.61      0.69       460
     Salmonella_enterica       0.45      0.63      0.52       482
   Staphylococcus_aureus       0.13      0.19      0.16       476
Streptococcus_pneumoniae       0.30      0.08      0.13       478
  Streptococcus_pyogenes       0.21      0.03      0.06       465

                accuracy                           0.36      4800
               macro avg       0.37      0.36      0.33      4800
            weighted avg       0.36      0.36      0.33      4800
 

		 BaggingClassifier 

                        

                          precision    recall  f1-score   support

    Bacteroides_fragilis       0.87      0.88      0.87       491
    Campylobacter_jejuni       0.95      0.88      0.91       489
      Enterococcus_hirae       0.84      0.88      0.86       486
        Escherichia_coli       0.81      0.82      0.81       488
  Escherichia_fergusonii       0.81      0.88      0.84       485
   Klebsiella_pneumoniae       0.97      0.87      0.92       460
     Salmonella_enterica       0.88      0.83      0.86       482
   Staphylococcus_aureus       0.82      0.94      0.88       476
Streptococcus_pneumoniae       0.88      0.83      0.85       478
  Streptococcus_pyogenes       0.87      0.88      0.87       465

                accuracy                           0.87      4800
               macro avg       0.87      0.87      0.87      4800
            weighted avg       0.87      0.87      0.87      4800
 

		 LabelPropagation 

                          precision    recall  f1

                          precision    recall  f1-score   support

    Bacteroides_fragilis       0.94      0.93      0.93       491
    Campylobacter_jejuni       0.95      0.96      0.95       489
      Enterococcus_hirae       0.87      0.92      0.89       486
        Escherichia_coli       0.88      0.91      0.90       488
  Escherichia_fergusonii       0.90      0.92      0.91       485
   Klebsiella_pneumoniae       0.95      0.97      0.96       460
     Salmonella_enterica       0.94      0.90      0.92       482
   Staphylococcus_aureus       0.92      0.94      0.93       476
Streptococcus_pneumoniae       0.92      0.87      0.90       478
  Streptococcus_pyogenes       0.92      0.88      0.90       465

                accuracy                           0.92      4800
               macro avg       0.92      0.92      0.92      4800
            weighted avg       0.92      0.92      0.92      4800
 

		 RidgeClassifier 

                          precision    recall  f1-